In [ ]:
#### INSTALLATION OF LIBRARIES
!sudo apt update
!sudo apt install openjdk-17-jdk -y
#!curl -JLO 'https://apache.osuosl.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz'
!tar xvf spark-3.3.1-bin-hadoop3.tgz
!mv spark-3.3.1-bin-hadoop3 /opt/spark
!pip install pyspark missingno findspark plotly wandb koalas pymysql

In [ ]:
#### LIBRARIES
import os
os.environ["JAVA_HOME"] = "/usr"
os.environ["SPARK_HOME"] = "/opt/spark"
#### BASIC
import pandas as pd

#### SPARK
from pyspark.sql import SparkSession
import pyspark.pandas as ps
spark = SparkSession.builder.master("local").getOrCreate()

spark.sparkContext.setLogLevel("OFF")
import sqlalchemy
ps.set_option('compute.ops_on_diff_frames', True)
from transform_funcs import *
#from tiny_functions import *

def lower_col_names(cols):
    new_names = {}
    for x in cols:
        new_names[x] = x.lower()
    return new_names    

In [2]:
import pandas as pd
import json
import ast
import numpy as np
import datetime
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.cluster import KMeans

def get_len(value):
  ls=value.split(', ')
  return len(ls)


def dicc(row):
    #result = json.loads(row)
    result = ast.literal_eval(row)
    return result

def etl_atributtes(business):
    atributtes = pd.json_normalize(data = business['attributes'])
    atributtes['BusinessParking'].fillna("{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}", inplace=True)
    atributtes.loc[atributtes['BusinessParking'] == 'None', 'BusinessParking'] = "{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}"

    atributtes.loc[atributtes['Ambience'] == 'None', 'Ambience'] = "{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': False}"
    atributtes['Ambience'] = atributtes['Ambience'].fillna("{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': False}")

    

    garage = pd.json_normalize(atributtes.BusinessParking.apply(dicc))
    ambience = pd.json_normalize(atributtes['Ambience'].apply(dicc))

    atributtes['garage'] = garage['garage']
    atributtes['garage'].fillna(0, inplace=True)

    atributtes.drop(['BusinessParking', 'Ambience'], axis=1, inplace=True)

    ambience['good_ambience'] = ambience['romantic'] + ambience['intimate'] + ambience['classy'] + ambience['hipster'] + ambience['touristy'] + ambience['trendy'] + ambience['upscale'] + ambience['casual'] + ambience['divey']

    ambience.loc[ambience['good_ambience'] > 1, 'good_ambience'] = 1
    ambience.fillna(0, inplace=True)
    atributtes['good_ambience'] = ambience['good_ambience']

    atributtes['RestaurantsTakeOut'].fillna(0, inplace=True)
    atributtes['RestaurantsDelivery'].fillna(0, inplace=True)
    atributtes['RestaurantsTakeOut'] = atributtes['RestaurantsTakeOut'].map({'True': 1, 'False': 0, 'None': 0})
    atributtes['RestaurantsDelivery'] = atributtes['RestaurantsDelivery'].map({'True': 1, 'False': 0, 'None': 0})
    atributtes['delivery'] = atributtes['RestaurantsTakeOut'] + atributtes['RestaurantsDelivery']
    atributtes['delivery'] = pd.to_numeric(atributtes['delivery'], errors='coerce').fillna(0).astype(int)
    atributtes['delivery'] = atributtes['delivery'].replace(to_replace=2, value=1)


    atributtes.drop(['RestaurantsTakeOut', 'RestaurantsDelivery'], axis=1, inplace=True)

    top20 = atributtes.notna().sum().sort_values(ascending=False).head(20).index.tolist()

    atributtes = atributtes[top20]

    atributtes.fillna(0, inplace=True)
    atributtes.replace('None', 0, inplace=True)
    atributtes.replace('False', 0, inplace=True)
    atributtes.replace(False, 0, inplace=True)
    atributtes.replace('True', 1, inplace=True)

    atributtes['WiFi'] = np.where(atributtes['WiFi'] != 0, 1, 0)

    atributtes.loc[atributtes['Alcohol'] == "u'none'", 'Alcohol'] = 0
    atributtes.loc[atributtes['Alcohol'] == "'none'", 'Alcohol'] = 0
    atributtes.loc[atributtes['Alcohol'] != 0, 'Alcohol'] = 1
    atributtes['Alcohol'].unique()

    atributtes.loc[atributtes['NoiseLevel'].str.contains('quiet').fillna(False), 'NoiseLevel'] = 1
    atributtes.loc[atributtes['NoiseLevel'].str.contains('average').fillna(False), 'NoiseLevel'] = 2
    atributtes.loc[atributtes['NoiseLevel'].str.contains('loud').fillna(False), 'NoiseLevel'] = 3
    atributtes.loc[atributtes['RestaurantsAttire'] != 0,'RestaurantsAttire'] = 1
    atributtes.loc[atributtes['garage'] != 0,'garage'] = 1
    atributtes.loc[atributtes['GoodForMeal'] == 0,'GoodForMeal'] = "{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'brunch': False, 'breakfast': False}"
    atributtes['GoodForMeal'].fillna("{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'brunch': False, 'breakfast': False}", inplace=True)
    meal_types = pd.json_normalize(atributtes['GoodForMeal'].apply(dicc))
    atributtes.drop(['GoodForMeal'], axis=1, inplace=True)
    atributtes['meal_diversity'] = meal_types['dessert'] + meal_types['latenight'] + meal_types['lunch'] + meal_types['dinner'] + meal_types['brunch'] + meal_types['breakfast']
    atributtes['meal_diversity'].fillna(0, inplace=True)
    atributtes['business_id'] = business['business_id']
    return atributtes

def open_time(x):
    if pd.isnull(x):
        return None
    else:
        return x.split('-')[0]

def close_time(x):
    if pd.isnull(x):
        return None
    else:
        return x.split('-')[1]


def normalize_hours(lista:list, df):
    for i in lista:
        df.loc[pd.notnull(df[i]), i] = pd.to_datetime(df.loc[pd.notnull(df[i]), i]) - pd.to_datetime(df.loc[pd.notnull(df[i]), i]).dt.normalize()

def mean_open_hour(lista:list, df):

    for i in range(df.shape[0]):
        temp_sum = []
        for j in lista:
            if pd.notnull(df.loc[i, j]):
                temp_sum.append(df.loc[i, j])
        if len(temp_sum) > 0:
            df.loc[i, 'mean_open_hour'] = sum(temp_sum,datetime.timedelta())/len(temp_sum)

def mean_close_hour(lista:list, df):
    for i in range(df.shape[0]):
        temp_sum = []
        for j in lista:
            if pd.notnull(df.loc[i, j]):
                temp_sum.append(df.loc[i, j])
        if len(temp_sum) > 0:
            df.loc[i, 'mean_close_hour'] = sum(temp_sum,datetime.timedelta())/len(temp_sum)


def etl_hours(business):
    hours = pd.json_normalize(data = business['hours'])
    hours['business_id'] = business['business_id']
    subset = hours[~pd.isnull(hours[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']]).any(1)].index
    hours['7days'] = 0
    hours.loc[subset, '7days'] = 1
    subset = hours[~pd.isnull(hours[['Friday', 'Saturday', 'Sunday']]).any(1)].index
    hours['weekends'] = 0
    hours.loc[subset, 'weekends'] = 1

    hours['monday_open_time'] = pd.to_datetime(hours['Monday'].apply(open_time), format='%H:%M')
    hours['monday_close_time'] = pd.to_datetime(hours['Monday'].apply(close_time), format='%H:%M')
    hours['monday_total_hours'] = (hours['monday_close_time'] - hours['monday_open_time'])/pd.Timedelta(hours=1)
    hours.loc[hours['monday_total_hours'] == 0.0, 'monday_total_hours'] = 24.0

    hours['tuesday_open_time'] = pd.to_datetime(hours['Tuesday'].apply(open_time), format='%H:%M')
    hours['tuesday_close_time'] = pd.to_datetime(hours['Tuesday'].apply(close_time), format='%H:%M')
    hours['tuesday_total_hours'] = (hours['tuesday_close_time'] - hours['tuesday_open_time'])/pd.Timedelta(hours=1)
    hours.loc[hours['tuesday_total_hours'] == 0.0, 'tuesday_total_hours'] = 24.0


    hours['wednesday_open_time'] = pd.to_datetime(hours['Wednesday'].apply(open_time), format='%H:%M')
    hours['wednesday_close_time'] = pd.to_datetime(hours['Wednesday'].apply(close_time), format='%H:%M')
    hours['wednesday_total_hours'] = (hours['wednesday_close_time'] - hours['wednesday_open_time'])/pd.Timedelta(hours=1)
    hours.loc[hours['wednesday_total_hours'] == 0.0, 'wednesday_total_hours'] = 24.0


    hours['thursday_open_time'] = pd.to_datetime(hours['Thursday'].apply(open_time), format='%H:%M')
    hours['thursday_close_time'] = pd.to_datetime(hours['Thursday'].apply(close_time), format='%H:%M')
    hours['thursday_total_hours'] = (hours['thursday_close_time'] - hours['thursday_open_time'])/pd.Timedelta(hours=1)
    hours.loc[hours['thursday_total_hours'] == 0.0, 'thursday_total_hours'] = 24.0


    hours['friday_open_time'] = pd.to_datetime(hours['Friday'].apply(open_time), format='%H:%M')
    hours['friday_close_time'] = pd.to_datetime(hours['Friday'].apply(close_time), format='%H:%M')
    hours['friday_total_hours'] = (hours['friday_close_time'] - hours['friday_open_time'])/pd.Timedelta(hours=1)
    hours.loc[hours['friday_total_hours'] == 0.0, 'friday_total_hours'] = 24.0


    hours['saturday_open_time'] = pd.to_datetime(hours['Saturday'].apply(open_time), format='%H:%M')
    hours['saturday_close_time'] = pd.to_datetime(hours['Saturday'].apply(close_time), format='%H:%M')
    hours['saturday_total_hours'] = (hours['saturday_close_time'] - hours['saturday_open_time'])/pd.Timedelta(hours=1)
    hours.loc[hours['saturday_total_hours'] == 0.0, 'saturday_total_hours'] = 24.0


    hours['sunday_open_time'] = pd.to_datetime(hours['Sunday'].apply(open_time), format='%H:%M')
    hours['sunday_close_time'] = pd.to_datetime(hours['Sunday'].apply(close_time), format='%H:%M')
    hours['sunday_total_hours'] = (hours['sunday_close_time'] - hours['sunday_open_time'])/pd.Timedelta(hours=1)
    hours.loc[hours['sunday_total_hours'] == 0.0, 'sunday_total_hours'] = 24.0

    lista_total_open_hours = ['monday_total_hours', 'tuesday_total_hours', 'wednesday_total_hours', 'thursday_total_hours', 'friday_total_hours', 'saturday_total_hours', 'sunday_total_hours']
    lista_open = ['monday_open_time', 'tuesday_open_time', 'wednesday_open_time', 'thursday_open_time', 'friday_open_time', 'saturday_open_time', 'sunday_open_time']
    lista_close = ['monday_close_time', 'tuesday_close_time', 'wednesday_close_time', 'thursday_close_time', 'friday_close_time', 'saturday_close_time', 'sunday_close_time']
    lista_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    lista_total =  lista_total_open_hours + lista_open + lista_close + lista_days

    hours['n_open_days'] = pd.notnull(hours[lista_open]).sum(axis=1)
    hours['mean_total_hours_open'] = hours[lista_total_open_hours].abs().mean(axis=1)

    normalize_hours(lista_open, hours)
    normalize_hours(lista_close, hours)
    mean_open_hour(lista_open, hours)
    mean_close_hour(lista_close, hours)
    hours.drop(lista_total, axis=1, inplace=True)
    return hours

def etl_categories(business):
    categories = business['categories'].str.split(', ', expand=True)
    categories.index = business['business_id']
    df = categories.T.stack().groupby('business_id').apply(list).reset_index(name='categories')
    mlb = MultiLabelBinarizer()
    df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('categories')),
                            columns=mlb.classes_,
                            index=df.index))
    df['total_categories'] = df.sum(axis=1)
    most_frequent = ['Restaurants',
    'Food',
    'Shopping',
    'Home Services',
    'Beauty & Spas',
    'Nightlife',
    'Health & Medical',
    'Local Services',
    'Bars',
    'Automotive', 'total_categories', 'business_id']
    df = df.fillna(0)
    df = df[most_frequent]
    return df

def etl_gps(business):
    for_clustering = business[['latitude', 'longitude', 'business_id']]
    kmeans = KMeans(n_clusters = 11, init ='k-means++')
    Y_axis = for_clustering['latitude'].values.reshape(-1,1)
    X_axis = for_clustering['longitude'].values.reshape(-1,1)
    for_clustering.loc[:,'areas'] = kmeans.fit_predict(X_axis, Y_axis)
    return for_clustering


def get_len(value):
  """
  It takes a string of comma separated names and returns the number of names in the string
  
  :param value: the value of the column you're applying the function to
  :return: The number of friends in the list.
  """
  ls = value.split(', ')
  return len(ls)



In [6]:
def drop_bad_str(input):
    if pd.isna(input):
        return 'NO DATA'
    else:
        output = input.strip().replace('  ',' ').lower()
        if len(output) <=2:
            return 'REMOVE_THIS_ROW'
        else: return output

In [7]:
import dateutil
def transform_dates(input):
    if isinstance(input,str):
        try:
            return(dateutil.parser.parse(input))  #NO REEMPLAZA POR MODA | USAR APPLY
        except: return pd.NaT
    else: return pd.NaT

In [55]:
def load_n_checkin():
    #### READS FILE AND MAKES TRANSFORMATION
    print('READING TIPS FILE')
    checkin = pd.read_json('./data/checkin.json', lines=True)

    #### TRANSFORMATIONS
    print('DROPPING DUPLICATED ROWS')
    checkin = checkin.drop_duplicates()

    print("CALCULATING TOTAL CHECKINS")
    checkin['total'] = checkin['date'].apply(lambda x: get_len(x))

    checkin.drop('date', axis=1, inplace=True)

    checkin.rename(columns=lower_col_names(checkin.columns), inplace=True)

    print('CONNECTING TO DATABASE and UPLOADING')
    
    engine = sqlalchemy.create_engine("mysql+pymysql://{user}:{pw}@{address}/{db}"
                .format(user="root",
                        address = '35.239.80.227:3306',
                        pw="Henry12.BORIS99",
                        db="yelp"))

    checkin.to_sql('n_checkins', con=engine, if_exists='replace', index=False)

    print(pd.read_sql("SELECT COUNT(*) FROM n_checkins;", con=engine).head())

load_n_checkin()

READING TIPS FILE
DROPPING DUPLICATED ROWS
CALCULATING TOTAL CHECKINS
CONNECTING TO DATABASE and UPLOADING
   COUNT(*)
0    131930


In [68]:
def load_tips():
    #### READS FILE AND MAKES TRANSFORMATION
    print('READING TIPS FILE')
    tip = pd.read_json('./data/tip_big/tip_big.json',lines=True)

    #### TRANSFORMATIONS
    print('DROPPING DUPLICATED ROWS')
    tip = tip.drop_duplicates()
    print('CLEANING STRINGS')
    tip['text'] = tip['text'].apply(lambda x: drop_bad_str(x))

    print('NORMALIZING DATES')
    tip['date'] = tip['date'].apply(lambda x: transform_dates(x)).dt.strftime('%Y-%m-%d')

    tip.rename(columns=lower_col_names(tip.columns), inplace=True)

    print('CONNECTING TO DATABASE and UPLOADING')
    
    engine = sqlalchemy.create_engine("mysql+pymysql://{user}:{pw}@{address}/{db}"
                .format(user="root",
                        address = '35.239.80.227:3306',
                        pw="Henry12.BORIS99",
                        db="yelp"))

    tip.to_sql('tip', con=engine, if_exists='replace', index=False)

    print(pd.read_sql("SELECT COUNT(*) FROM tip;", con=engine).head())

load_tips()

READING TIPS FILE
DROPPING DUPLICATED ROWS
CLEANING STRINGS
NORMALIZING DATES


/usr/local/lib/python3.9/dist-packages/pandas/core/indexes/extension.py:101: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(result, name=self.name)


CONNECTING TO DATABASE and UPLOADING
   COUNT(*)
0    899854


In [72]:
def load_top_tips():
    ##### UPLOADS SMALL DATASET TOP TIPS TO CASSANDRA
    #### READS FILE AND MAKES TRANSFORMATION
    print('READING TIPS FILE')
    tip = pd.read_json('./data/tip_big/tip_big.json',lines=True)

    #### MAKES PANDAS TRANSFORMATION TO GET TOP TIPS
    top_tips = pd.DataFrame(tip['business_id'].value_counts())
    top_tips.rename(columns={'business_id': 'number_tips'}, inplace=True)
    top_tips['business_id'] = top_tips.index
    top_tips.reset_index(drop=True, inplace=True)

    top_tips.rename(columns=lower_col_names(top_tips.columns), inplace=True)

    print('CONNECTING TO DATABASE and UPLOADING')

    engine = sqlalchemy.create_engine("mysql+pymysql://{user}:{pw}@{address}/{db}"
                .format(user="root",
                        address = '35.239.80.227:3306',
                        pw="Henry12.BORIS99",
                        db="yelp"))

    top_tips.to_sql('top_tip', con=engine, if_exists='replace', index=False)

    print(pd.read_sql("SELECT COUNT(*) FROM top_tip;", con=engine).head())

load_top_tips()


READING TIPS FILE
CONNECTING TO DATABASE and UPLOADING
   COUNT(*)
0    105376


In [75]:
from functools import reduce

def load_business():
    print('READING BUSINESS FILE')
    business = pd.read_json('./data/business.json', lines=True)
    

    print('TRANSFORMING ATTRIBUTES')
    atributtes = etl_atributtes(business)

    print('TRANSFORMING HOURS')
    hours = etl_hours(business)

    print('TRANSFORMING CATEGORIES') #FILLLED NA
    categories = etl_categories(business)

    print('GPS CLUSTERING')
    gps = etl_gps(business)

    print('FIXING RestaurantsPriceRange2 AND DELIVERY')
    atributtes['RestaurantsPriceRange2'] = pd.to_numeric(atributtes['RestaurantsPriceRange2'])
    
    print(atributtes['delivery'].unique())

    print('MERGING DATAFRAMES')
    data_frames = [business, atributtes, categories, hours, gps]
    full_data = reduce(lambda left,right: pd.merge(left,right,on='business_id', how='left'), data_frames)
    full_data = full_data.drop(['attributes', 'hours', 'city', 'state', 'categories', 'latitude_y', 'longitude_y'], axis=1)

    full_data.rename(columns={'Home Services':'HomeServices','Beauty & Spas':'BeautyAndSpas', 'Health & Medical':'HealthAndMedical','Local Services':'LocalServices', '7days':'SevenDays'}, inplace=True)

    full_data['mean_open_hour'] = full_data.mean_open_hour.astype(str)
    full_data['mean_close_hour'] = full_data.mean_close_hour.astype(str)
    full_data['RestaurantsPriceRange2'] = full_data.RestaurantsPriceRange2.astype(str)

    full_data.rename(columns=lower_col_names(full_data.columns), inplace=True)

    print('DONE')

    print('CONNECTING TO DATABASE and UPLOADING')

    engine = sqlalchemy.create_engine("mysql+pymysql://{user}:{pw}@{address}/{db}"
                .format(user="root",
                        address = '35.239.80.227:3306',
                        pw="Henry12.BORIS99",
                        db="yelp"))

    full_data.to_sql('business_clean', con=engine, if_exists='replace', index=False)

    print(pd.read_sql("SELECT COUNT(*) FROM business_clean;", con=engine).head())


load_business()

READING BUSINESS FILE
TRANSFORMING ATTRIBUTES
TRANSFORMING HOURS
TRANSFORMING CATEGORIES


/tmp/ipykernel_3867/1686793685.py:196: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df['total_categories'] = df.sum(axis=1)


GPS CLUSTERING


/tmp/ipykernel_3867/1686793685.py:216: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  for_clustering.loc[:,'areas'] = kmeans.fit_predict(X_axis, Y_axis)


FIXING RestaurantsPriceRange2 AND DELIVERY
[0 1]
MERGING DATAFRAMES
DONE
CONNECTING TO DATABASE and UPLOADING
   COUNT(*)
0    150346


In [80]:
def load_user():
    print('READING USER FILE')
    user = ps.read_json('./data/user.json', lines=True)
    print('DROPPING DUPLICATED ROWS')
    user = user.drop_duplicates()

    print('NORMALIZING DATES')
    user['yelping_since'] = user['yelping_since'].apply(transform_dates).dt.strftime('%Y-%m-%d')

    print('DROPPING ELITE & FRIENDS')
    user = user.drop(['friends', 'elite'], axis=1)

    print('USER COLS')
    print(user.columns)

    user.rename(columns=lower_col_names(user.columns), inplace=True)

    
    l1 = [0,200000, 400000, 800000, 1000000,1200000, 1400000, 1600000, 1800000]
    l2 = [200000, 400000, 800000, 1000000,1200000, 1400000, 1600000, 1800000, user.shape[0]]

    print('CONNECTING MYSQL')
    engine = sqlalchemy.create_engine("mysql+pymysql://{user}:{pw}@{address}/{db}"
        .format(user="root",
                address = '35.239.80.227:3306',
                pw="Henry12.BORIS99",
                db="yelp"))

    for i,j in list(zip(l1,l2)):        
        df = user.iloc[i:j, :].to_pandas()
        print('UPLOADING PART {}'.format(i))
        df.to_sql('user', con=engine, if_exists='append', index=False)
    print(pd.read_sql("SELECT COUNT(*) FROM business_clean;", con=engine).head())

load_user()

READING USER FILE


/opt/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_json`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


DROPPING DUPLICATED ROWS
NORMALIZING DATES


DROPPING ELITE & FRIENDS
USER COLS
Index(['average_stars', 'compliment_cool', 'compliment_cute',
       'compliment_funny', 'compliment_hot', 'compliment_list',
       'compliment_more', 'compliment_note', 'compliment_photos',
       'compliment_plain', 'compliment_profile', 'compliment_writer', 'cool',
       'fans', 'funny', 'name', 'review_count', 'useful', 'user_id',
       'yelping_since'],
      dtype='object')


CONNECTING MYSQL


...

UPLOADING PART 1400000


/opt/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas DataFrame is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


UPLOADING PART 1600000


/opt/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas DataFrame is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


UPLOADING PART 1800000
   COUNT(*)
0    150346


In [ ]:
def load_user_metrics(): 
    """
    The function takes a dataframe of users and returns a dataframe with the influencer score for each
    user

    :param user: the user dataframe
    :return: A dataframe with the columns: n_interactions_received, n_interactions_send, fans,
    friends_number, Score_influencer, Influencer, user_id
    """
    print('READING USER FILE')
    user = ps.read_json('./data/user.json', lines=True)
    print('DROPPING DUPLICATED ROWS')
    user = user.drop_duplicates()

    print('GENERATING INTERACTIONS RECIEVED COLUMN')
    user['n_ints_rec'] = user[[ 'compliment_hot',
    'compliment_more', 'compliment_profile', 'compliment_cute',
    'compliment_list', 'compliment_note', 'compliment_plain',
    'compliment_cool', 'compliment_funny', 'compliment_writer',
    'compliment_photos']].sum(axis=1)
    print('GENERATING INTERACTIONS SEND COLUMN')
    user['n_interactions_send'] = user['useful'] + user['funny'] + user['cool']
    print('GENERATING FRIENDS NUM COLUMN')
    user['friends_number'] = user.friends.apply(get_len)
    print('GENERATING INFLUENCER COLUMN')
    user['Influencer'] = user['n_ints_rec'] / (1 + user['friends_number'] + user['fans'])
    user['Influencer'].fillna(0, inplace = True)
    print('GENERATING INFLUENCER SCORE COLUMN')
    user['Influencer_Score'] = 1 - (1 / (1 + user['Influencer']))
    print('GENERATING INFLUENCER 2 COLUMN')
    user['Influencer_2'] = user['n_ints_rec'] / (1 + user['fans'])
    user['Influencer_2'].fillna(0, inplace = True)
    print('GENERATING INFLUENCER SCORE 2 COLUMN')
    user['Influencer_Score_2'] = 1 - (1 / (1 + user['Influencer_2']))

    print('GENERATING NEW DF')
    user = user[['user_id', 'n_ints_rec', 'n_interactions_send', 'fans', 'friends_number',
    'Influencer', 'Influencer_Score', 'Influencer_2', 'Influencer_Score_2']]
    print('TRANSFORMATIONS FOR USER METRICS DONE')

    user.rename(columns=lower_col_names(user.columns), inplace=True)

    print('CONNECTING TO DATABASE and UPLOADING')

    l1 = [0,200000, 400000, 800000, 1000000,1200000, 1400000, 1600000, 1800000]
    l2 = [200000, 400000, 800000, 1000000,1200000, 1400000, 1600000, 1800000, user.shape[0]]

    print('CONNECTING MYSQL')
    engine = sqlalchemy.create_engine("mysql+pymysql://{user}:{pw}@{address}/{db}"
        .format(user="root",
                address = '35.239.80.227:3306',
                pw="Henry12.BORIS99",
                db="yelp"))

    for i,j in list(zip(l1,l2)):        
        df = user.iloc[i:j, :].to_pandas()
        print('UPLOADING PART {}'.format(i))
        df.to_sql('user_metrics', con=engine, if_exists='append', index=False)
    print(pd.read_sql("SELECT COUNT(*) FROM user_metrics;", con=engine).head())

load_user_metrics()

In [10]:
def load_user_metrics(): 
    """
    The function takes a dataframe of users and returns a dataframe with the influencer score for each
    user

    :param user: the user dataframe
    :return: A dataframe with the columns: n_interactions_received, n_interactions_send, fans,
    friends_number, Score_influencer, Influencer, user_id
    """
    print('READING USER FILE')
    user = ps.read_json('./data/user.json', lines=True).iloc[1800000:, :]
    print('DROPPING DUPLICATED ROWS')
    user = user.drop_duplicates()

    print('GENERATING INTERACTIONS RECIEVED COLUMN')
    user['n_ints_rec'] = user[[ 'compliment_hot',
    'compliment_more', 'compliment_profile', 'compliment_cute',
    'compliment_list', 'compliment_note', 'compliment_plain',
    'compliment_cool', 'compliment_funny', 'compliment_writer',
    'compliment_photos']].sum(axis=1)
    print('GENERATING INTERACTIONS SEND COLUMN')
    user['n_interactions_send'] = user['useful'] + user['funny'] + user['cool']
    print('GENERATING FRIENDS NUM COLUMN')
    user['friends_number'] = user.friends.apply(get_len)
    print('GENERATING INFLUENCER COLUMN')
    user['Influencer'] = user['n_ints_rec'] / (1 + user['friends_number'] + user['fans'])
    user['Influencer'].fillna(0, inplace = True)
    print('GENERATING INFLUENCER SCORE COLUMN')
    user['Influencer_Score'] = 1 - (1 / (1 + user['Influencer']))
    print('GENERATING INFLUENCER 2 COLUMN')
    user['Influencer_2'] = user['n_ints_rec'] / (1 + user['fans'])
    user['Influencer_2'].fillna(0, inplace = True)
    print('GENERATING INFLUENCER SCORE 2 COLUMN')
    user['Influencer_Score_2'] = 1 - (1 / (1 + user['Influencer_2']))

    print('GENERATING NEW DF')
    user = user[['user_id', 'n_ints_rec', 'n_interactions_send', 'fans', 'friends_number',
    'Influencer', 'Influencer_Score', 'Influencer_2', 'Influencer_Score_2']]
    print('TRANSFORMATIONS FOR USER METRICS DONE')

    user.rename(columns=lower_col_names(user.columns), inplace=True)

    print('CONNECTING TO DATABASE and UPLOADING')

    print('CONNECTING MYSQL')
    engine = sqlalchemy.create_engine("mysql+pymysql://{user}:{pw}@{address}/{db}"
        .format(user="root",
                address = '35.239.80.227:3306',
                pw="Henry12.BORIS99",
                db="yelp"))

    user.to_pandas().to_sql('user_metrics', con=engine, if_exists='append', index=False)
    print(pd.read_sql("SELECT COUNT(*) FROM user_metrics;", con=engine).head())

load_user_metrics()

READING USER FILE


/opt/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_json`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


DROPPING DUPLICATED ROWS
GENERATING INTERACTIONS RECIEVED COLUMN


GENERATING INTERACTIONS SEND COLUMN
GENERATING FRIENDS NUM COLUMN


GENERATING INFLUENCER COLUMN
GENERATING INFLUENCER SCORE COLUMN
GENERATING INFLUENCER 2 COLUMN
GENERATING INFLUENCER SCORE 2 COLUMN
GENERATING NEW DF
TRANSFORMATIONS FOR USER METRICS DONE
CONNECTING TO DATABASE and UPLOADING
CONNECTING MYSQL


/opt/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas DataFrame is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


   COUNT(*)
0   1987897


In [6]:
def load_review():

    print('CONNECTING TO DATABASE and UPLOADING')
    engine = sqlalchemy.create_engine("mysql+pymysql://{user}:{pw}@{address}/{db}"
    .format(user="root",
            address = '35.239.80.227:3306',
            pw="Henry12.BORIS99",
            db="yelp"))

    lista1 = []
    lista2 = []
    for i in range(0,6600000, 100000):
        lista1.append(i)

    lista2 = lista1[1:]
    lista2.append(6815608)

    splitter = list(zip(lista1,lista2))

    #print(splitter)
    for i,j in splitter:        

        print('READING REVIEW FILE {}+{}'.format(i,j))
        review = ps.read_json('./data/review_big/review_big.json').iloc[i:j, :]
        
        print('DROPPING DUPLICATED ROWS')
        review = review.drop_duplicates()

        #print('NORMALIZING DATES')
        #review['date'] = review['date'].apply(transform_dates).dt.strftime('%Y-%m-%d')

        review.rename(columns=lower_col_names(review.columns), inplace=True)

        print('UPLOADING PART {}{}'.format(i,j))
        review.to_pandas().to_sql('review', con=engine, if_exists='append', index=False)
        
    print(pd.read_sql("SELECT COUNT(*) FROM review;", con=engine).head())
    

load_review()

CONNECTING TO DATABASE and UPLOADING
READING REVIEW FILE 0+100000


/usr/local/lib/python3.9/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_json`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


DROPPING DUPLICATED ROWS
UPLOADING PART 0100000


/usr/local/lib/python3.9/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas DataFrame is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


READING REVIEW FILE 100000+200000


...

DROPPING DUPLICATED ROWS
UPLOADING PART 64000006500000


READING REVIEW FILE 6500000+6815608


/usr/local/lib/python3.9/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_json`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
/usr/local/lib/python3.9/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas DataFrame is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


DROPPING DUPLICATED ROWS
UPLOADING PART 65000006815608


   COUNT(*)
0   6815608


In [5]:
!pip install pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 11.5 MB/s eta 0:00:00


In [7]:
def load_sentiment_by_business():

    print('CONNECTING TO DATABASE and UPLOADING')
    engine = sqlalchemy.create_engine("mysql+pymysql://{user}:{pw}@{address}/{db}"
    .format(user="root",
            address = '35.239.80.227:3306',
            pw="Henry12.BORIS99",
            db="yelp"))


    print('READING FILE')
    sentiment = pd.read_csv('./data/sentiment/sentiment_ok_unique.csv')

    print('DROPPING DUPLICATED ROWS')
    sentiment = sentiment.drop_duplicates()

    sentiment.to_sql('sentiment_by_business', con=engine, if_exists='append', index=False)
        
    print(pd.read_sql("SELECT COUNT(*) FROM sentiment_by_business;", con=engine).head())
    

load_sentiment_by_business()

CONNECTING TO DATABASE and UPLOADING
READING FILE
DROPPING DUPLICATED ROWS
   COUNT(*)
0    150346


In [8]:
import glob

In [9]:
def load_sentiment_by_review():

    print('CONNECTING TO DATABASE and UPLOADING')
    engine = sqlalchemy.create_engine("mysql+pymysql://{user}:{pw}@{address}/{db}"
    .format(user="root",
            address = '35.239.80.227:3306',
            pw="Henry12.BORIS99",
            db="yelp"))

    path = './data/sentiment/ok/'
    all_csv = glob.glob(path + "/*.csv")

    for i in all_csv:
        print('READING FILE: ', i)
        sentiment = pd.read_csv(i)
        print('LOADING TO DATABASE')
        sentiment.to_sql('sentiment_by_review', con=engine, if_exists='append', index=False)
        
    print(pd.read_sql("SELECT COUNT(*) FROM sentiment_by_review;", con=engine).head())
    

load_sentiment_by_review()

CONNECTING TO DATABASE and UPLOADING
READING FILE:  ./data/sentiment/ok/dataset_1.csv
LOADING TO DATABASE
READING FILE:  ./data/sentiment/ok/dataset_3.csv
LOADING TO DATABASE
READING FILE:  ./data/sentiment/ok/dataset_7.csv
LOADING TO DATABASE
READING FILE:  ./data/sentiment/ok/dataset_5.csv
LOADING TO DATABASE
READING FILE:  ./data/sentiment/ok/dataset_4.csv
LOADING TO DATABASE
READING FILE:  ./data/sentiment/ok/dataset_2.csv
LOADING TO DATABASE
READING FILE:  ./data/sentiment/ok/dataset_6.csv
LOADING TO DATABASE
   COUNT(*)
0   6990280


In [10]:
def load_review_timeseries():

    print('CONNECTING TO DATABASE and UPLOADING')
    engine = sqlalchemy.create_engine("mysql+pymysql://{user}:{pw}@{address}/{db}"
    .format(user="root",
            address = '35.239.80.227:3306',
            pw="Henry12.BORIS99",
            db="yelp"))


    print('READING FILE')
    review_ts = pd.read_csv('./data/review_timeseries.csv')

    print('DROPPING DUPLICATED ROWS')
    review_ts = review_ts.drop_duplicates()

    review_ts.to_sql('review_timeseries_top_brands', con=engine, if_exists='append', index=False)
        
    print(pd.read_sql("SELECT COUNT(*) FROM review_timeseries_top_brands;", con=engine).head())
    

load_review_timeseries()

CONNECTING TO DATABASE and UPLOADING
READING FILE
DROPPING DUPLICATED ROWS
   COUNT(*)
0    133399


In [11]:
def load_tip_timeseries():

    print('CONNECTING TO DATABASE and UPLOADING')
    engine = sqlalchemy.create_engine("mysql+pymysql://{user}:{pw}@{address}/{db}"
    .format(user="root",
            address = '35.239.80.227:3306',
            pw="Henry12.BORIS99",
            db="yelp"))


    print('READING FILE')
    tip_ts = pd.read_csv('./data/tip_timeseries.csv')

    print('DROPPING DUPLICATED ROWS')
    tip_ts = tip_ts.drop_duplicates()

    tip_ts.to_sql('tip_timeseries_top_brands', con=engine, if_exists='append', index=False)
        
    print(pd.read_sql("SELECT COUNT(*) FROM tip_timeseries_top_brands;", con=engine).head())
    

load_tip_timeseries()

CONNECTING TO DATABASE and UPLOADING
READING FILE
DROPPING DUPLICATED ROWS
   COUNT(*)
0     24519


In [12]:
def load_checkin_timeseries():

    print('CONNECTING TO DATABASE and UPLOADING')
    engine = sqlalchemy.create_engine("mysql+pymysql://{user}:{pw}@{address}/{db}"
    .format(user="root",
            address = '35.239.80.227:3306',
            pw="Henry12.BORIS99",
            db="yelp"))


    print('READING FILE')
    checkin_ts = pd.read_csv('./data/checkin_timeseries.csv')

    print('DROPPING DUPLICATED ROWS')
    checkin_ts = checkin_ts.drop_duplicates()

    checkin_ts.to_sql('checkin_timeseries_top_brands', con=engine, if_exists='append', index=False)
        
    print(pd.read_sql("SELECT COUNT(*) FROM checkin_timeseries_top_brands;", con=engine).head())
    

load_checkin_timeseries()

CONNECTING TO DATABASE and UPLOADING
READING FILE
DROPPING DUPLICATED ROWS
   COUNT(*)
0    500150


In [15]:
def load_attributes():

    print('CONNECTING TO DATABASE and UPLOADING')
    engine = sqlalchemy.create_engine("mysql+pymysql://{user}:{pw}@{address}/{db}"
    .format(user="root",
            address = '35.239.80.227:3306',
            pw="Henry12.BORIS99",
            db="yelp"))


    print('READING FILE')
    attributes_ts = pd.read_csv('./data/attributes.csv', low_memory=False)

    #print('DROPPING DUPLICATED ROWS')
    #attributes_ts = attributes_ts.drop_duplicates()

    attributes_ts.to_sql('attributes_business', con=engine, if_exists='append', index=False)
        
    print(pd.read_sql("SELECT COUNT(*) FROM attributes_business;", con=engine).head())
    

load_attributes()

CONNECTING TO DATABASE and UPLOADING
READING FILE


/tmp/ipykernel_32/419694690.py:12: DtypeWarning: Columns (35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  attributes_ts = pd.read_csv('./data/attributes.csv')


   COUNT(*)
0    150346


In [24]:
def load_categories():

    print('CONNECTING TO DATABASE and UPLOADING')
    engine = sqlalchemy.create_engine("mysql+pymysql://{user}:{pw}@{address}/{db}"
    .format(user="root",
            address = '35.239.80.227:3306',
            pw="Henry12.BORIS99",
            db="yelp"))

    print('READING FILE')
    business = pd.read_json('./data/business.json', lines=True)
    categories_ts = business[['business_id','categories']]

    #print('DROPPING DUPLICATED ROWS')
    #categories_ts = categories_ts.drop_duplicates()

    categories_ts.to_sql('categories_business', con=engine, if_exists='append', index=False)
        
    print(pd.read_sql("SELECT COUNT(*) FROM categories_business;", con=engine).head())
    

load_categories()

CONNECTING TO DATABASE and UPLOADING
READING FILE
   COUNT(*)
0    150346


In [25]:
len(str('mWMc6_wTdE0EUBKIGXDVfA'))

22

### DROP TABLE FUNCTION for MySQL 

In [13]:
# from sqlalchemy import MetaData
# from sqlalchemy import create_engine
# from sqlalchemy.engine.url import URL
# from sqlalchemy.ext.declarative import declarative_base

# engine = sqlalchemy.create_engine("mysql+pymysql://{user}:{pw}@{address}/{db}"
# .format(user="root",
#         address = '35.239.80.227:3306',
#         pw="Henry12.BORIS99",
#         db="yelp"))
# def drop_table(table_name, engine=engine):
#     Base = declarative_base()
#     metadata = MetaData()
#     metadata.reflect(bind=engine)
#     table = metadata.tables[table_name]
#     if table is not None:
#         Base.metadata.drop_all(engine, [table], checkfirst=True)

# drop_table('categories_business')

In [1]:
import sqlalchemy
import pandas as pd

In [3]:
def load_business_target():

    print('CONNECTING TO DATABASE and UPLOADING')
    engine = sqlalchemy.create_engine("mysql+pymysql://{user}:{pw}@{address}/{db}"
    .format(user="root",
            address = '35.239.80.227:3306',
            pw="Henry12.BORIS99",
            db="yelp"))


    print('READING FILE')
    business_target = pd.read_csv('../../../ML/data/target_3_influencer_modified.csv', low_memory=False)

    #print('DROPPING DUPLICATED ROWS')
    #attributes_ts = attributes_ts.drop_duplicates()

    business_target.to_sql('business_target', con=engine, if_exists='append', index=False)
        
    print(pd.read_sql("SELECT COUNT(*) FROM business_target;", con=engine).head())
    

load_business_target()

CONNECTING TO DATABASE and UPLOADING
READING FILE
   COUNT(*)
0    150346
